In [13]:
# import libs

import numpy as np
import Laser
import random


# Important definition
positive angle in CCW direction

# Define functions

In [14]:
# read G-Code file and return a list of strings
def read_gcode_file(filename):
    tmp = []
    with open(filename, 'r') as f:
        gcode = f.readlines()
    for line in gcode:
        tmp.append(line)
    
    return np.array(tmp)
    

In [15]:
def parseCoord(linestring):
    line = linestring
    X = None
    Y = None
    Z = None

    # check the command of the gcode line (string until first space)
    command = line.split(' ')[0]
    
    # check if line contains ; which means it is a comment and skip line
    if not ';' in line:
        # parse the X Y Z coords from string to float
        if 'X' in line:
            X = float(line.split('X')[-1].split(' ')[0])
        if 'Y' in line:
            Y = float(line.split('Y')[-1].split(' ')[0])
        if 'Z' in line:
            Z = float(line.split('Z')[-1].split(' ')[0])
    else:
        X = None
        Y = None
        Z = None
    return X, Y, Z, command, linestring


In [16]:
# calculate the distance between two points
def calcDistance(x0, y0, x1, y1):
    return np.sqrt((x1-x0)**2 + (y1-y0)**2)


In [17]:
# calculate angle between two points (atan2)
def calcAngle(x0, y0, x1, y1):
    newAngle = np.arctan2((y1-y0),(x1-x0))
    if newAngle < 0:
        newAngle = 2*np.pi - np.abs(newAngle)
    # convert to degrees
    newAngle = newAngle*180/np.pi
    return newAngle

In [18]:
def findClosestAngle(targetAngle, LaserObj):
    currentAngles = LaserObj.getAngles()
    

In [19]:
def postProcessGcode(gCodeArray, nLaser):


    return True

# Test functions

In [20]:
# test calcAngle function
if (0):
    print(calcAngle(0,0,1,1))
    print(calcAngle(0,0,1,-1))
    print(calcAngle(0,0,-1,-1))
    print(calcAngle(0,0,-1,1))


In [21]:
# test the parseCoord function
if(1):

# read the gcode file
    gcode = read_gcode_file("GCODE.txt")

    # create empty lists for the coordinates
    X = []
    Y = []
    Z = []
    cmd = []
    i = 0
    # loop over all lines in the gcode file
    for line in gcode:
        #print(i)
        #i += 1
        # parse the coordinates from the line
        x, y, z, command, linestring = parseCoord(line)
        # append the coordinates to the lists
        X.append(x)
        Y.append(y)
        Z.append(z)
        cmd.append(command)

# print first 10 elements of X Y Z
j = 50
if(1):
    for i in range(j):
        print(f"X: {X[i]} Y: {Y[i]} Z: {Z[i]} cmd: {cmd[i]}")

X: None Y: None Z: None cmd: ;FLAVOR:Marlin

X: None Y: None Z: None cmd: ;TIME:1701

X: None Y: None Z: None cmd: ;Filament
X: None Y: None Z: None cmd: ;Layer
X: None Y: None Z: None cmd: ;MINX:71.8

X: None Y: None Z: None cmd: ;MINY:91.8

X: None Y: None Z: None cmd: ;MINZ:0.2

X: None Y: None Z: None cmd: ;MAXX:178.2

X: None Y: None Z: None cmd: ;MAXY:118.2

X: None Y: None Z: None cmd: ;MAXZ:5

X: None Y: None Z: None cmd: ;Generated
X: None Y: None Z: None cmd: M82
X: None Y: None Z: None cmd: G21
X: None Y: None Z: None cmd: G90
X: None Y: None Z: None cmd: M82
X: None Y: None Z: None cmd: M104
X: None Y: None Z: None cmd: M140
X: None Y: None Z: None cmd: M190
X: None Y: None Z: None cmd: M109
X: None Y: None Z: None cmd: G28
X: None Y: None Z: None cmd: G80
X: None Y: None Z: None cmd: G92
X: None Y: None Z: None cmd: G1
X: None Y: None Z: None cmd: G1
X: None Y: None Z: None cmd: G1
X: None Y: None Z: None cmd: G92
X: None Y: None Z: None cmd: G92
X: None Y: None Z: None cm

# Main 

In [24]:
# read the gcode file
gcode = read_gcode_file("GCODE.txt")

# print shape of gcode
print(gcode.shape)

# create laser object
numberOfLaser = 4
laser = Laser.Laser(numberOfLaser)

laser.update_angles(105)
print(laser.getAngles())
print(laser.getNormAngles())


(10045,)
[105. 195. 285. 375.]
[105. 195. 285.  15.]
